# Задача 5 (бонусная): экспериментальное исследование алгоритмов для регулярных запросов

## Подготовка окружения

Для исполнения кода, приведённого в данном исследовании, необходима система с ядром Linux, подготовленная при помощи выполнения команд, приведённых далее.

In [107]:
# @formatter:off
!pip install -r ../requirements.txt
!pip install pycubool matplotlib
# @formatter:on

## Введение

В данном работе рассматривается задача достижимости в графе с регулярными ограничениями и исследуется производительность её решений матричными подходами на языке Python при помощи библиотек, осуществляющих матричные операции на CPU и GPU.

### О задаче достижимости

Пусть дан конечный ориентированный помеченный граф $G = (V, E, L)$, в котором каждому пути $\pi \in E^*$, состоящему из рёбер с метками $l_1, ..., l_n \in L$, сопоставляется слово по правилу $\omega(\pi) = l_1 * ... * l_n$, где $*$ --- конкатенация.

Пусть также дано регулярное выражение $R$ для языка над алфавитом $L$.

В данных условиях мы будем рассматривать три варианта задачи о достижимости в графе $G$ с регулярными ограничениями, заданным $R$.

#### Постановка 1. Достижимость между всеми парами вершин.

Данная постановка задачи является наиболее общей.

В ней задача заключается в нахождении таких пар вершин $G$, что между ними существует путь, которому сопоставляется слово из языка, задаваемого $R$. То есть задача заключается в нахождении множества $\{(v_i, v_j) \in V^2: \exists \pi \in E^*: start(\pi) = v_i, final(\pi) = v_j, \omega(\pi) \in language(R)\}$.

#### Постановка 2. Достижимость для всего множества заданных вершин.

В данной постановке задачи из множества $V$ всех вершин графа дополнительно выделяются подмножества стартовых и финальных вершин: $V_S \subseteq V$ и $V_F \subseteq V$ соответственно. Задача заключается в нахождении всех таких финальных вершин, таких, что до них существуют пути из стартовых, которым сопоставляются слова из языка, задаваемого $R$. То есть задача заключается в нахождении множества $\{v_i \in V_F: \exists \pi \in E^*: start(\pi) \in V_S, final(\pi) = v_j, \omega(\pi) \in language(R)\}$.

#### Постановка 3. Достижимость для каждой вершины из заданного множества стартовых вершин.

Данная постановка является комбинацией предыдущих двух. Здесь также выделяются множества стартовых и финальных вершин $V_S и $V_F, в которых найденные пути должны начинаться и оканчиваться соответственно, однако, аналогично постановке 1, требуется найти множество пар вершин-начал и вершин-концов данных путей. То есть задача заключается в нахождении множества $\{(v_i, v_j) \in V_S \times V_F: \exists \pi \in E^*: start(\pi) = v_i, final(\pi) = v_j, \omega(\pi) \in language(R)\}$.

### Исследуемые решения

В данной работе для решения поставленных выше задач используются методы, кратко описываемые следующим образом:

1. **Достижимость между всеми парами вершин**. Через тензорное произведение строится пересечение конечных автоматов, соответствующих графу и регулярному выражению, для которого затем считается транзитивное замыкание через матричное возведение в степень.
2. **Достижимость для всего множества заданных вершин**. Алгоритм решения основывается на синхронном обходе в ширину конечных автоматов, соответствующих регулярному выражению и графу. Обход проводится через умножение фронта, представленного как матрица условно разделенная на соответствующие две части, на матрицу смежности, построенную как прямая сумма матриц смежности обоих автоматов. Матрица фронта в левой части представляет из себя единичную матрицу, а в правой --- строки-фронты-обхода, возможные для соответствующей вершины в левой части. После каждого умножения матрица фронта трансформируется для поддержки данного инварианта.
3. **Достижимость для каждой вершины из заданного множества стартовых вершин**. Принцип решения схож с предыдущим пунктом, однако фронт разделяется для каждой стартовой вершины и представляется как единая матрица, что позволяет хранить информацию о стартовой вершине для каждой достигнутой финальной.

На практике матрицы смежности, используемые в представленных алгоритмах являются сильно разреженными, поэтому при их реализации становится выгодно представлять такие матрицы специальным образом, избегая хранения "пустых" ячеек. Каждый из описанных алгоритмов был реализован на языке Python при помощи двух библиотек, содержащих таких типы матриц:

1. [**scipy.sparse**](https://docs.scipy.org/doc/scipy/reference/sparse.html) --- модуль SciPy, предоставляющий несколько видов двухмерных разреженных массивов с интерфейсом взаимодействия, приближенным к таковому в NumPy. Все операции, проводимые с массивами, происходят на CPU.
2. [**pycubool**](https://github.com/JetBrains-Research/cuBool) --- обертка для C-библиотеки, позволяющая проводить операции линейной алгебры над разреженными матрицами на GPU марки NVIDIA при помощи технологии CUDA.

Оба способа реализации максимально идентичны (на сколько это позволяют интерфейсы представленных библиотек).

#### Реализация на scipy.sparse

Реализация алгоритмов, основанная на `scipy.sparse`, расположена в модуле [`project.rpq`](../project/rpq.py) данного репозитория.

In [108]:
from project.rpq import BfsMode
from project.rpq import rpq_by_bfs
from project.rpq import rpq_by_tensor

#### Реализация на pycubool

Реализация алгоритмов, основанная на `pycubool`, представленная ниже:

In [109]:
from itertools import product
from typing import Any
from typing import NamedTuple

import pycubool as cb
from pyformlang.finite_automaton import EpsilonNFA


class BoolDecompositionCuda:
    class StateInfo(NamedTuple):
        data: Any
        is_start: bool
        is_final: bool

    def __init__(
        self,
        states: list[StateInfo] | None = None,
        adjs: dict[Any, cb.Matrix] | None = None,
    ):
        self.states: list[BoolDecompositionCuda.StateInfo] = (
            states if states is not None else []
        )
        self.adjs: dict[Any, cb.Matrix] = adjs if adjs is not None else {}

    @classmethod
    def from_nfa(
        cls, nfa: EpsilonNFA, sort_states: bool = False
    ) -> "BoolDecompositionCuda":
        # Construct states, removing duplicates
        states = list(
            set(
                cls.StateInfo(
                    data=st.value,
                    is_start=st in nfa.start_states,
                    is_final=st in nfa.final_states,
                )
                for st in nfa.states
            )
        )
        if sort_states:
            states = sorted(states, key=lambda st: st.data)

        # Construct adjacency matrices
        adjs = {}
        transitions = nfa.to_dict()
        n = len(states)
        for n_from in transitions:
            for symbol, ns_to in transitions[n_from].items():
                adj = adjs.setdefault(symbol.value, cb.Matrix.empty((n, n)))
                beg_index = next(i for i, s in enumerate(states) if s.data == n_from)
                for n_to in ns_to if isinstance(ns_to, set) else {ns_to}:
                    end_index = next(i for i, s in enumerate(states) if s.data == n_to)
                    adj[beg_index, end_index] = True

        return cls(states, adjs)

    def intersect(self, other: "BoolDecompositionCuda") -> "BoolDecompositionCuda":
        # Set of states of the intersection is the product of the given sets of states
        # State is "start" if both of its element states are "start"
        # State is "final" if both of its element states are "final"
        states = [
            self.StateInfo(
                data=(st1.data, st2.data),
                is_start=st1.is_start and st2.is_start,
                is_final=st1.is_final and st2.is_final,
            )
            for st1, st2 in product(self.states, other.states)
        ]

        # Set of symbols of the intersection is the union of the given sets of symbols
        # Adjacency matrix for a symbol is a kronecker product of the given matrices of
        # this symbol
        adjs = {}
        n = len(states)
        for symbol in set(self.adjs.keys()).union(set(other.adjs.keys())):
            if symbol in self.adjs and symbol in other.adjs:
                adjs[symbol] = self.adjs[symbol].kronecker(other.adjs[symbol])
            else:
                adjs[symbol] = cb.Matrix.empty((n, n))

        return BoolDecompositionCuda(states, adjs)

    def transitive_closure_any_symbol(self) -> tuple[list[int], list[int]]:
        # Gather all matrices to get all existing paths
        n = len(self.states)
        adj_all = cb.Matrix.empty((n, n))
        for adj in self.adjs.values():
            adj_all = adj_all.ewiseadd(adj)

        # Transitive closure by repeated-squaring-like approach
        while True:
            # nvals gives the number of paths
            prev_path_num = adj_all.nvals
            # Multiplication gives new paths, while sum retains the old ones
            adj_all.mxm(adj_all, out=adj_all, accumulate=True)
            # If no new paths appear, all paths have been discovered
            if prev_path_num == adj_all.nvals:
                break

        # Convert to a more user-friendly representation
        return adj_all.to_lists()

    def _direct_sum(self, other: "BoolDecompositionCuda") -> "BoolDecompositionCuda":
        states = self.states + other.states

        adjs = {}
        for symbol in set(self.adjs.keys()).intersection(set(other.adjs.keys())):
            dsum = cb.Matrix.empty((len(states), len(states)))
            for i, j in self.adjs[symbol]:
                dsum[i, j] = True
            for i, j in other.adjs[symbol]:
                dsum[len(self.states) + i, len(self.states) + j] = True
            adjs[symbol] = dsum

        return BoolDecompositionCuda(states, adjs)

    def constrained_bfs(
        self, constraint: "BoolDecompositionCuda", separated: bool = False
    ) -> set[int] | set[tuple[int, int]]:
        # Save states number because will use them heavily for matrix construction
        n = len(constraint.states)
        m = len(self.states)

        direct_sum = constraint._direct_sum(self)

        # Create initial front from starts of constraint (left) and self (right)
        start_states_indices = [i for i, st in enumerate(self.states) if st.is_start]
        init_front = (
            _init_bfs_front(self.states, constraint.states)
            if not separated
            else _init_separated_bfs_front(
                self.states, constraint.states, start_states_indices
            )
        )

        # Create visited, fill with zeroes instead of init_front to get rid of initial
        # positions in the result
        visited = cb.Matrix.empty(init_front.shape)

        # Perform matrix-multiplication-based-BFS until visited stops changing
        while True:
            old_visited_nvals = visited.nvals

            # Perform a BFS step for each matrix in direct sum
            for _, adj in direct_sum.adjs.items():
                # Compute new front for the symbol
                front_part = (
                    visited.mxm(adj) if init_front is None else init_front.mxm(adj)
                )
                # Transform the resulting front so that:
                # 1. It only contains rows with non-zeros in both parts.
                # 2. Its left part contains non-zeroes only on its main diagonal.
                visited = visited.ewiseadd(_transform_front_part(front_part, n))

            # Can use visited instead now
            init_front = None

            # If no new non-zero elements have appeared, we've visited all we can
            if visited.nvals == old_visited_nvals:
                break

        # If visited a final self-state in final constraint-state, we found a result
        results = set()
        for i, j in visited.to_list():
            # Check that the element is from the self part (which is the main BFS part)
            # and the final state requirements are satisfied
            if j >= n and constraint.states[i % n].is_final:  # % is for separated BFS
                self_st_index = j - n
                if self.states[self_st_index].is_final:
                    results.add(
                        self_st_index
                        if not separated
                        else (start_states_indices[i // n], self_st_index)
                    )
        return results


def _init_bfs_front(
    self_states: list[BoolDecompositionCuda.StateInfo],
    constr_states: list[BoolDecompositionCuda.StateInfo],
    self_start_indices: list[int] | None = None,
) -> cb.Matrix:
    front = cb.Matrix.empty((len(constr_states), len(constr_states) + len(self_states)))

    if self_start_indices is None:
        self_start_indices = [j for j, st in enumerate(self_states) if st.is_start]

    for i, st in enumerate(constr_states):
        if st.is_start:
            front[i, i] = True  # Mark diagonal element as start
            for j in self_start_indices:
                front[i, len(constr_states) + j] = True  # Fill start row

    return front


def _init_separated_bfs_front(
    self_states: list[BoolDecompositionCuda.StateInfo],
    constr_states: list[BoolDecompositionCuda.StateInfo],
    start_states_indices: list[int],
) -> cb.Matrix:
    fronts = [
        _init_bfs_front(
            self_states,
            constr_states,
            self_start_indices=[st_i],
        )
        for st_i in start_states_indices
    ]

    if len(fronts) == 0:
        return cb.Matrix.empty(
            (len(constr_states), len(constr_states) + len(self_states))
        )

    # Build the united front as a vertical stack of the separated fronts
    result = cb.Matrix.empty(
        (len(fronts) * len(constr_states), len(constr_states) + len(self_states))
    )
    vstack_helper = cb.Matrix.empty((len(fronts), 1))
    for i, front in enumerate(fronts):
        vstack_helper.build(rows={i}, cols={0})
        result = result.ewiseadd(vstack_helper.kronecker(front))
    return result


def _transform_front_part(front_part: cb.Matrix, constr_states_num: int) -> cb.Matrix:
    transformed_front_part = cb.Matrix.empty(front_part.shape)
    # Perform the transformation by rows
    for i, j in front_part.to_list():
        # If the element is from the constraint part
        if j < constr_states_num:
            non_zero_row_right = front_part[i: i + 1, constr_states_num:]
            # If the right part contains non-zero elements
            if non_zero_row_right.nvals > 0:
                # Account for separated front
                row_shift = i // constr_states_num * constr_states_num
                # Mark the row in the left part
                transformed_front_part[row_shift + j, j] = True
                # Update right part of the row
                for _, r_j in non_zero_row_right:
                    transformed_front_part[
                        row_shift + j, constr_states_num + r_j
                    ] = True
    return transformed_front_part

In [110]:
import networkx as nx
import pyformlang.regular_expression as re
from typing import TypeVar

from project.automata_utils import graph_to_nfa
from project.automata_utils import regex_to_min_dfa

_NodeType = TypeVar("_NodeType")


def rpq_by_tensor_cuda(
    graph: nx.Graph,
    query: str | re.Regex,
    starts: set[_NodeType] | None = None,
    finals: set[_NodeType] | None = None,
) -> set[tuple[_NodeType, _NodeType]]:
    # Create boolean decompositions for the graph and the query
    graph_decomp = BoolDecompositionCuda.from_nfa(graph_to_nfa(graph, starts, finals))
    query_decomp = BoolDecompositionCuda.from_nfa(regex_to_min_dfa(query))

    # Intersection of decompositions gives intersection of languages
    intersection = graph_decomp.intersect(query_decomp)
    # Transitive closure helps determine reachability
    transitive_closure_indices = intersection.transitive_closure_any_symbol()

    # Two nodes satisfy the query if one is the beginning of a path (i.e. a word) and
    # the other is its end
    results = set()
    for n_from_i, n_to_i in zip(*transitive_closure_indices):
        n_from = intersection.states[n_from_i]
        n_to = intersection.states[n_to_i]
        if n_from.is_start and n_to.is_final:
            beg_graph_node = n_from.data[0]
            end_graph_node = n_to.data[0]
            results.add((beg_graph_node, end_graph_node))
    return results


def rpq_by_bfs_cuda(
    graph: nx.Graph,
    query: str | re.Regex,
    starts: set[_NodeType] | None = None,
    finals: set[_NodeType] | None = None,
    mode: BfsMode = BfsMode.FIND_COMMON_REACHABLE_SET,
) -> set[_NodeType] | set[tuple[_NodeType, _NodeType]]:
    graph_decomp = BoolDecompositionCuda.from_nfa(graph_to_nfa(graph, starts, finals))
    query_decomp = BoolDecompositionCuda.from_nfa(regex_to_min_dfa(query))

    result_indices = graph_decomp.constrained_bfs(
        query_decomp, separated=mode == BfsMode.FIND_REACHABLE_FOR_EACH_START
    )

    match mode:
        case BfsMode.FIND_COMMON_REACHABLE_SET:
            return {graph_decomp.states[i].data for i in result_indices}
        case BfsMode.FIND_REACHABLE_FOR_EACH_START:
            return {
                (graph_decomp.states[i].data, graph_decomp.states[j].data)
                for i, j in result_indices
            }



## Цель работы

Целью данной работы является анализ производительности представленных решений задачи достижимости в графе с регулярными ограничениями в трёх её постановках, описанных выше, а именно, ответ на следующие три вопроса:

1. При каких условиях использование `pycubool` даёт выигрыш в производительности по сравнению с `scipy.sparse`? Для каждой из трёх постановок задачи.
2. При использовании `pycubool`, начиная с какого размера стартового множества выгоднее решать задачу для всех пар и выбирать нужные?
3. При использовании `pycubool`, насколько решение задачи во второй её постановке быстрее решения в третьей при одинаковых начальных условиях?

## Условия эксперимента

В данной секции описывается каким образом проводился эксперимент, соответствующий цели данной работы.

### Характеристики оборудование

Оборудование, на котором проводились замеры, обладает следующими характеристиками:

In [111]:
# @formatter:off

! echo '----- OS  -----'
! hostnamectl | grep -E 'Operating System|Kernel'

! echo '----- CPU -----'
! lscpu | grep -E 'Architecture|Model name|Thread\(s\)|Core\(s\)|Socket\(s\)|MHz|cache'

! echo '----- GPU -----'
! nvidia-smi -L
! nvidia-smi | grep CUDA

! echo '----- RAM -----'
! free -m

# @formatter:on

----- OS  -----
Operating System: Ubuntu 22.04.1 LTS
          Kernel: Linux 5.15.0-50-generic
----- CPU -----
Architecture:                    x86_64
Model name:                      AMD Ryzen 5 5600X 6-Core Processor
Thread(s) per core:              2
Core(s) per socket:              6
Socket(s):                       1
CPU max MHz:                     4650.2920
CPU min MHz:                     2200.0000
L1d cache:                       192 KiB (6 instances)
L1i cache:                       192 KiB (6 instances)
L2 cache:                        3 MiB (6 instances)
L3 cache:                        32 MiB (1 instance)
----- GPU -----
GPU 0: NVIDIA GeForce RTX 2060 (UUID: GPU-8294f82e-bdf0-92ae-58e4-939c50d2a707)
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
----- RAM -----
               total        used        free      shared  buff/cache   available
Mem:           15920        7718        7469          14         732        7906
Swap:              0

### Описание использованного набора данных

Для проведения экспериментов необходимо было выбрать графы для поиска путей и регулярные выражения для задания ограничений.

#### Выбор графов

Эксперимент проводился на следующих десяти графах из [датасета `CFPQ_Data`](https://jetbrains-research.github.io/CFPQ_Data/dataset/index.html):

- `skos`, `atom`, `bzip`, `pr`, `ls`, `pathways`, `enzyme`, `apache` --- как репрезентативные по количеству вершин и рёбер относительно всего датасета
- `go_hierarchy`, `geospecies` --- как имеющие сравнительно высокое отношение числа рёбер к числу вершин

Графы с числом вершин или рёбер, приближающимся к трём миллионам и выше, решено было не рассматривать, так как время их обработки оказалось достаточно велико.

In [112]:
from project.graph_utils import load_graph_from_cfpq_data

GRAPHS: list[nx.Graph] = []


def init_graphs():
    for name in [
        "skos",
        "atom",
        "bzip",
        "pr",
        "funding",
        "ls",
        "pizza",
        "gzip",
        "pathways",
        "enzyme",
    ]:
        graph = load_graph_from_cfpq_data(name)
        graph.name = name
        GRAPHS.append(graph)


init_graphs()

[2022-10-12 10:31:41]>INFO>Found graph with name='skos'
[2022-10-12 10:31:42]>INFO>Load archive graph_archive=PosixPath('/home/eyjafl/.venv/formlang/lib/python3.10/site-packages/cfpq_data/data/skos.tar.gz')
[2022-10-12 10:31:42]>INFO>Unzip graph name='skos' to file graph=PosixPath('/home/eyjafl/.venv/formlang/lib/python3.10/site-packages/cfpq_data/data/skos/skos.csv')
[2022-10-12 10:31:42]>INFO>Remove archive graph_archive=PosixPath('/home/eyjafl/.venv/formlang/lib/python3.10/site-packages/cfpq_data/data/skos.tar.gz')
[2022-10-12 10:31:42]>INFO>Load graph=<networkx.classes.multidigraph.MultiDiGraph object at 0x7f14379edd20> from path=PosixPath('/home/eyjafl/.venv/formlang/lib/python3.10/site-packages/cfpq_data/data/skos/skos.csv')
[2022-10-12 10:31:42]>INFO>Found graph with name='atom'
[2022-10-12 10:31:42]>INFO>Load archive graph_archive=PosixPath('/home/eyjafl/.venv/formlang/lib/python3.10/site-packages/cfpq_data/data/atom.tar.gz')
[2022-10-12 10:31:42]>INFO>Unzip graph name='atom' t

Ниже приведена статистика по выбранным графам.

In [113]:
import pandas as pd
from collections import Counter
from project.graph_utils import get_graph_info


def get_most_common(source: list[str], n: int) -> list[tuple[str, int]]:
    return sorted(Counter(source).most_common(n), key=lambda x: (x[1], x[0]), reverse=True)


def graph_stats() -> pd.DataFrame:
    stats = []

    for graph in GRAPHS:
        info = get_graph_info(graph)
        stats.append([
            graph.name,
            info.nodes_num,
            info.edges_num,
            info.edges_num / info.nodes_num,
            get_most_common(info.labels, 4),
        ])

    return pd.DataFrame(
        sorted(stats, key=lambda st: st[2]),
        columns=["Name", "Nodes", "Edges", "Edges per node", "Labels (most common -> least common)"],
    )


pd.set_option("display.max_colwidth", None)
graph_stats()

,Name,Nodes,Edges,Edges per node,Labels (most common -> least common)
0,skos,144,252,1.750000,"[(type, 70), (label, 32), (isDefinedBy, 32), (definition, 32)]"
1,atom,291,425,1.460481,"[(type, 138), (label, 129), (subClassOf, 122), (comment, 11)]"
2,bzip,632,556,0.879747,"[(d, 297), (a, 259)]"
3,pr,815,692,0.849080,"[(d, 359), (a, 333)]"
4,funding,778,1086,1.395887,"[(type, 304), (label, 231), (comment, 229), (subClassOf, 90)]"
5,ls,1687,1453,0.861292,"[(d, 750), (a, 703)]"
6,pizza,671,1980,2.950820,"[(disjointWith, 398), (type, 365), (subClassOf, 259), (onProperty, 188)]"
7,gzip,2687,2293,0.853368,"[(d, 1218), (a, 1075)]"
8,pathways,6238,12363,1.981885,"[(type, 3118), (subClassOf, 3117), (label, 3117), (narrower, 3010)]"
9,enzyme,48815,86543,1.772877,"[(type, 14989), (comment, 11954), (altLabel, 10088), (subClassOf, 8163)]"


#### Выбор регулярных выражений

Для каждого графа создавалось по четыре регулярных выражения следующих видов:

1. $R_1 = l_1 \ | \ l_2$ --- одно из наиболее простых выражений.
2. $R_2 = (l_1 \ | \ l_2) \ (l_3 \ | \ l_4)$ --- более сложное выражение.
3. $R_3 = l_1 \ l_2 \ l_3 \ l_4$ --- выражение со сравнительно большим числом состояний и малым числом переходов.
4. $R_4 = (l_1 \ | \ l_2 \ | \ l_3 \ | \ l_4)^* \ l_3 \ l_4$ --- выражение со сравнительно большим числом переходов и малым числом состояний.

В качестве меток $l_1, l_2, l_3, l_4$ выбирались четыре наиболее часто встречающиеся в конкретном графе метки в порядке убывания количества рёбер с ними (а если некоторые метки появляются на одинаковом количестве рёбер, то они сортируются в обратном алфавитном порядке). Если в графе менее четырёх меток, то существующие метки брались повторно в том же порядке необходимое количество раз. То есть, если граф обладает метками `ab` и `ac` на трёх рёбрах и `b` --- на одном, то $l_1 =$`ac`, $l_2 =$`ab`, $l_3 =$`b`, $l_4 =$`ac`.

Далее представлен код для генерации описанных регулярных выражений.

In [114]:
import pyformlang.finite_automaton as fa

REGEX_LABELS_NUM = 4


def label_to_regexp(l: str) -> re.Regex:
    re0 = re.Regex("")
    re0.head = fa.Symbol(l)
    return re0


def r1(l1: str, l2: str, _l3: str, _l4: str) -> re.Regex:
    del _l3, _l4
    re1 = label_to_regexp(l1)
    re2 = label_to_regexp(l2)
    return re1.union(re2)


def r2(l1: str, l2: str, l3: str, l4: str) -> re.Regex:
    re1 = label_to_regexp(l1)
    re2 = label_to_regexp(l2)
    re12 = re1.union(re2)

    re3 = label_to_regexp(l3)
    re4 = label_to_regexp(l4)
    re34 = re3.union(re4)

    return re12.concatenate(re34)


def r3(l1: str, l2: str, l3: str, l4: str) -> re.Regex:
    re1 = label_to_regexp(l1)
    re2 = label_to_regexp(l2)
    re3 = label_to_regexp(l3)
    re4 = label_to_regexp(l4)
    return re1.concatenate(re2).concatenate(re3).concatenate(re4)


def r4(l1: str, l2: str, l3: str, l4: str) -> re.Regex:
    re1 = label_to_regexp(l1)
    re2 = label_to_regexp(l2)
    re3 = label_to_regexp(l3)
    re4 = label_to_regexp(l4)
    re1234 = re1.union(re2).union(re3).union(re4)
    re1234 = re1234.kleene_star()

    return re1234.concatenate(re3).concatenate(re4)


REGEXES = {
    "l1 | l2": r1,
    "(l1 | l2) (l3 | l4)": r2,
    "l1 l2 l3 l4": r3,
    "(l1 | l2 | l3 | l4)* l3 l4": r4
}


def create_regexes_for_graph(graph: nx.Graph) -> list[re.Regex]:
    info = get_graph_info(graph)
    labels = get_most_common(info.labels, REGEX_LABELS_NUM)
    while len(labels) < REGEX_LABELS_NUM:
        labels += labels
    labels = labels[:REGEX_LABELS_NUM]

    return [r(*labels) for r in REGEXES.values()]

Также ниже приведены характеристики данных регулярных выражений как конечных автоматов, к которым они будут сводиться в исследуемых алгоритмах.

In [115]:
def regex_stats() -> pd.DataFrame:
    stats = []

    for s, r in [(s, r("a", "b", "c", "d")) for s, r in REGEXES.items()]:
        dfa = regex_to_min_dfa(r)
        stats.append([
            s,
            len(dfa.symbols),
            len(dfa.states),
            dfa.get_number_transitions(),
            dfa.get_number_transitions() / len(dfa.states),
        ])

    return pd.DataFrame(stats, columns=["Regex", "Symbols", "States", "Transitions", "Transitions per state"])


regex_stats()

,Regex,Symbols,States,Transitions,Transitions per state
0,l1 | l2,2,2,2,1.000000
1,(l1 | l2) (l3 | l4),4,3,4,1.333333
2,l1 l2 l3 l4,4,5,4,0.800000
3,(l1 | l2 | l3 | l4)* l3 l4,4,3,12,4.000000


#### Выбор множеств стартовых и финальных вершин

Стоит отметить, что множество стартовых вершин может заметно и нетривиально повлиять на быстродействие алгоритмов, решающих задачу в постановках 2 и 3, так как если из выбранных вершин достижима малая область графа, то обход завершится быстрее. Для алгоритма в постановке задачи под номером 3 число стартовых вершин также определяет размер матрицы-фронта. Для обеспечения воспроизводимости результатов эксперимента множества стартовых вершин генерировалось при помощи функций [`generate_multiple_source`](https://jetbrains-research.github.io/CFPQ_Data/reference/graphs/generated/cfpq_data.graphs.utils.multiple_source_utils.html#cfpq_data.graphs.utils.multiple_source_utils.generate_multiple_source) и [`generate_multiple_source_percent`](https://jetbrains-research.github.io/CFPQ_Data/reference/graphs/generated/cfpq_data.graphs.utils.multiple_source_utils.html#cfpq_data.graphs.utils.multiple_source_utils.generate_multiple_source_percent) из библиотеки `CFPQ_Data` со значением параметра `seed`, равным `42`. Так как для вопроса 2 из цели данной работы требуется изучить влияние размера множества стартовых вершин, то значение параметра `set_size` определяется конкретным экспериментом.

Напротив, множество финальных вершин влияет на быстродействие всех трёх алгоритмов тривиально и лишь своим размером: во всех случаях на завершающем этапе достигнутые вершины проверяются на принадлежность к множеству финальных вершин, что происходит через поиск по множеству. Поэтому для простоты в качестве множества финальных вершин выбиралось всё множество вершин графа.

In [116]:
# Seed to generate the set of start nodes
START_SET_SEED = 42

### Проводимые замеры

Для ответа на каждый из поставленных вопросов нужно провести соответствующие замеры.

#### Замеры для вопроса 1 (о выгоде от использования `pycubool`)

Для выяснения того, при каких условиях использование `pycubool` даёт выигрыш в производительности против использования `scipy.sparse`, нужно выявить основные характеристики матриц, которые влияют на скорость выполнения данными библиотеками матричных операций.

Естественно предположить, что это размер и степень разреженности (мера того, насколько матрица заполнена непустыми элементами): чем больше каждая из этих величин, тем больше нетривиальных операций нужно провести. Среди параметров графа и регулярного выражения на эти величины влияют, соответственно, число вершин (или же состояний конечного автомата, в случае регулярного выражения) и отношение числа рёбер (или же переходов) к числу вершин (или же состояний).

Таким образом, для каждого из трёх алгоритмов следует для реализаций на `scipy.sparse` и `pycubool` замерить скорость их работы на всевозможных комбинациях выбранных графов и регулярных выражений с фиксированными стартовыми множествами и проследить зависимость разницы данных скоростей от указанных параметров.

#### Замеры для вопроса 2 (о влиянии размера стартового множества на выгоду использования решений первой и третьей формулировок)

Чтобы понять, при каких размерах множества стартовых вершинах, используя `pycubool`, эффективнее решать задачу в её первой постановке (то есть искать достижимые вершины среди всех пар вершин, а затем отбирать нужные), а при каких --- в третьей (то есть искать достижимые вершины лишь от стартовых вершин), требуется замерить скорость работы данных решений, реализованных на `pycubool`, при разных размерах стартового множества вершин.

Таким образом, следует замерить скорость работы реализаций решения первой и третьей постановок задачи на `pycubool` на всевозможных комбинациях выбранных графов и регулярных выражений с различными размерами стартового множества и проследить, какая из реализаций будет показывать лучший результат в зависимости от взятых размеров.

#### Замеры для вопроса 3 (о сравнении быстродействия решений второй и третьей формулировок)

Чтобы выяснить, насколько решение задачи на `pycubool` в её второй формулировке (поиск для всего стартового множества вместе) быстрее решения в третьей (поиск для каждой стартовой вершины отдельно) при равных начальных условиях нужно замерить время их работы на одинаковых графах и регулярных запросах с равными стартовыми множествами.

Таким образом, следует замерить скорость работы реализаций решения второй и третьей постановок задачи на `pycubool` на всевозможных комбинациях выбранных графов и регулярных выражений с некоторыми фиксированным стартовыми множествами и сравнить результаты.

#### Итог по необходимым замерам

Итого, для ответа на все три вопроса достаточно провести следующие замеры времени выполнения:

- на всевозможных комбинациях выбранных графов и регулярных выражений с фиксированными стартовыми множествами --- для всех реализаций;
- на всевозможных комбинациях выбранных графов, регулярных выражений и размеров стартовых множеств --- для реализаций первой и третьей формулировок задачи на `pycubool`.

Для небольших графов (с числом вершин не более 10000) время работы замеряется пять раз подряд для получения среднего значения и стандартного отклонения. Для больших графов (с числом вершин более 10000) замеры проводятся по одному разу, так как время, затрачиваемое на выполнение алгоритмов на таких графах существенно превышает получаемые погрешности. Время замеряется при помощи модуля [`timeit`](https://docs.python.org/3/library/timeit.html).

In [117]:
# Percent of nodes to take as start in the fixed-start experiment
FIXED_START_SIZE = 50
# Sizes of start sets in varied-start experiment
VARIED_START_SIZES = [50, 100, 200, 400, 800, 1500, 3000, 6000, 10000]
# Number of consecutive times each measurement is run for small graphs
RUN_TIMES_SMALL = 5
# Number of consecutive times each measurement is run for huge graphs
RUN_TIMES_HUGE = 1


# Determines which graphs are considered small
def is_small(graph: nx.Graph) -> bool:
    return graph.number_of_edges() <= 10000

Ниже представлен код для проведения описанных замеров.

In [118]:
import signal
from timeit import repeat
from statistics import fmean
from statistics import stdev
from typing import Callable
from cfpq_data import generate_multiple_source


class RunTimeoutError(Exception):
    pass


def timeout_handler(_signum, _frame):
    print("Time out", end=" ")
    raise RunTimeoutError


def run_timed(f: Callable, run_times: int, timeout_each_s: int = 600):
    timeout_total_s = timeout_each_s * run_times

    signal.signal(signal.SIGALRM, timeout_handler)
    try:
        signal.alarm(timeout_total_s)
        times = repeat(f, repeat=run_times, number=1)
    except RunTimeoutError:
        times = [float("inf")]
    finally:
        signal.alarm(0)

    m = fmean(times)
    sd = stdev(times, m) if len(times) > 1 else 0.0
    return m, sd


def run_fixed_starts():
    results = []

    for g_i, graph in enumerate(GRAPHS, start=1):
        regexes = create_regexes_for_graph(graph)
        run_times = RUN_TIMES_SMALL if is_small(graph) else RUN_TIMES_HUGE
        starts = generate_multiple_source(graph, FIXED_START_SIZE, seed=START_SET_SEED)
        print(f"########## Graph {graph.name} ({g_i} / {len(GRAPHS)}, run times = {run_times}) ##########")

        graph_results = []
        for r_i, regex in enumerate(regexes, start=1):
            print(f"========== Regex #{r_i} ==========")

            print("by_tensor", end=" ")
            by_tensor = run_timed(
                lambda: rpq_by_tensor(graph, regex, starts=starts),
                run_times=run_times,
            )
            print(by_tensor)

            print("by_bfs_all", end=" ")
            by_bfs_all = run_timed(
                lambda: rpq_by_bfs(graph, regex, starts=starts, mode=BfsMode.FIND_COMMON_REACHABLE_SET),
                run_times=run_times,
            )
            print(by_bfs_all)

            print("by_bfs_each", end=" ")
            by_bfs_each = run_timed(
                lambda: rpq_by_bfs(graph, regex, starts=starts, mode=BfsMode.FIND_REACHABLE_FOR_EACH_START),
                run_times=run_times,
            )
            print(by_bfs_each)

            print("by_tensor_cuda", end=" ")
            by_tensor_cuda = run_timed(
                lambda: rpq_by_tensor_cuda(graph, regex, starts=starts),
                run_times=run_times,
            )
            print(by_tensor_cuda)

            print("by_bfs_all_cuda", end=" ")
            by_bfs_all_cuda = run_timed(
                lambda: rpq_by_bfs_cuda(graph, regex, starts=starts, mode=BfsMode.FIND_COMMON_REACHABLE_SET),
                run_times=run_times,
            )
            print(by_bfs_all_cuda)

            print("by_bfs_each_cuda", end=" ")
            by_bfs_each_cuda = run_timed(
                lambda: rpq_by_bfs_cuda(graph, regex, starts=starts, mode=BfsMode.FIND_REACHABLE_FOR_EACH_START),
                run_times=run_times,
            )
            print(by_bfs_each_cuda)

            graph_results.append(
                ((by_tensor, by_bfs_all, by_bfs_each), (by_tensor_cuda, by_bfs_all_cuda, by_bfs_each_cuda))
            )

        results.append(graph_results)

    return results


def run_varied_starts():
    results = []

    for g_i, graph in enumerate(GRAPHS, start=1):
        run_times = RUN_TIMES_SMALL if is_small(graph) else RUN_TIMES_HUGE
        regexes = create_regexes_for_graph(graph)
        print(f"########## Graph {graph.name} ({g_i} / {len(GRAPHS)}, run times = {run_times}) ##########")

        graph_results = []
        for r_i, regex in enumerate(regexes, start=1):
            print(f"========== Regex #{r_i} ==========")

            regex_results = {}
            for start_size in VARIED_START_SIZES:
                if start_size > graph.number_of_nodes():
                    continue
                starts = generate_multiple_source(graph, start_size, seed=START_SET_SEED)
                print(f"********** Start size #{start_size} **********")

                print("by_tensor_cuda", end=" ")
                by_tensor_cuda = run_timed(
                    lambda: rpq_by_tensor_cuda(graph, regex, starts=starts),
                    run_times=run_times,
                )
                print(by_tensor_cuda)

                print("by_bfs_each_cuda", end=" ")
                by_bfs_each_cuda = run_timed(
                    lambda: rpq_by_bfs_cuda(graph, regex, starts=starts, mode=BfsMode.FIND_REACHABLE_FOR_EACH_START),
                    run_times=run_times,
                )
                print(by_bfs_each_cuda)

                regex_results[start_size] = (by_tensor_cuda, by_bfs_each_cuda)

            graph_results.append(regex_results)

        results.append(graph_results)

    return results


fixed_starts_res = run_fixed_starts()
varied_starts_res = run_varied_starts()

[2022-10-12 10:31:45]>INFO>Generate set of source vertices of 50 nodes for graph=<networkx.classes.multidigraph.MultiDiGraph object at 0x7f14379edd20> for multiple-source evaluation


########## Graph skos (1 / 10, run times = 5) ##########
========== Regex #1 ==========
by_tensor (0.014972562399998423, 0.004765909359586005)
by_bfs_all (0.01130077820016595, 9.78897951922371e-05)
by_bfs_each (0.026348340600088706, 0.0002958505523572621)
by_tensor_cuda (0.008163136000257509, 0.00010699599853147703)
by_bfs_all_cuda (0.007894651800052088, 7.508099790222317e-05)
by_bfs_each_cuda (0.026781841399861152, 0.003770869742658866)
========== Regex #2 ==========
by_tensor (0.013214736599911703, 0.00017415656083704154)
by_bfs_all (0.01154623279999214, 4.9182234985349624e-05)
by_bfs_each (0.02656910040004732, 0.00030119672923772516)
by_tensor_cuda (0.008428205799646094, 7.08594594838355e-05)
by_bfs_all_cuda (0.008034014199984086, 3.172289211247255e-05)
by_bfs_each_cuda (0.025169092200303566, 0.0007971925499149784)
========== Regex #3 ==========
by_tensor (0.013262169400150015, 0.00014599623050018067)
by_bfs_all (0.011426847800066753, 4.641760238234392e-05)
by_bfs_each (0.0266772287

[2022-10-12 10:31:47]>INFO>Generate set of source vertices of 50 nodes for graph=<networkx.classes.multidigraph.MultiDiGraph object at 0x7f14395abd00> for multiple-source evaluation


(0.025863951200153678, 0.0008376230491187759)
########## Graph atom (2 / 10, run times = 5) ##########
========== Regex #1 ==========
by_tensor (0.029267133800021838, 0.0018417817327810286)
by_bfs_all (0.027812038599950028, 0.0019397127875403722)
by_bfs_each (0.04239768980005465, 0.0004619944065371219)
by_tensor_cuda (0.023219143000096663, 0.0011669798157614287)
by_bfs_all_cuda (0.02222218499991868, 0.0004124354634257992)
by_bfs_each_cuda (0.0426395060001596, 0.00864516684629376)
========== Regex #2 ==========
by_tensor (0.028353036599946792, 0.00027688771643494167)
by_bfs_all (0.026826543600327567, 0.00015081824484474495)
by_bfs_each (0.04249992340010067, 0.0002231980455927104)
by_tensor_cuda (0.022742284599917183, 4.954802345235434e-05)
by_bfs_all_cuda (0.022312946599959103, 0.00011354294847918989)
by_bfs_each_cuda (0.039280458399844065, 0.0008659741260555877)
========== Regex #3 ==========
by_tensor (0.028422193000005792, 0.000431087802877981)
by_bfs_all (0.026519784799893385, 3.709

[2022-10-12 10:31:51]>INFO>Generate set of source vertices of 50 nodes for graph=<networkx.classes.multidigraph.MultiDiGraph object at 0x7f14379edf00> for multiple-source evaluation


...

[2022-10-12 14:35:00]>INFO>Generate set of source vertices of 200 nodes for graph=<networkx.classes.multidigraph.MultiDiGraph object at 0x7f14379ee590> for multiple-source evaluation


(475.6007995680011, 0.0)
********** Start size #200 **********
by_tensor_cuda (476.0742227459996, 0.0)
by_bfs_each_cuda 

[2022-10-12 14:50:55]>INFO>Generate set of source vertices of 400 nodes for graph=<networkx.classes.multidigraph.MultiDiGraph object at 0x7f14379ee590> for multiple-source evaluation


(479.138158466998, 0.0)
********** Start size #400 **********
by_tensor_cuda 

KeyboardInterrupt: 

## Результаты эксперимента

В данном разделе демонстрируются результаты проведённых замеров и, исходя из них, даются ответы на поставленные ранее вопросы.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

ALGO_NAMES = ["Tensor multiplication", "United BFS", "Separated BFS"]

### Результаты по вопросу 1 (о выгоде от использования `pycubool`)

In [ ]:
def show_plots_1(
    res: list[list[tuple[tuple[tuple[float, float], tuple[float, float], tuple[float, float]], tuple[
        tuple[float, float], tuple[float, float], tuple[float, float]]]]]
):
    fig_size = (2.5 * len(REGEXES), len(GRAPHS))
    label_locations = np.arange(len(REGEXES))
    bar_width = 0.2
    cap_size = 4

    for algo_i, algo_name in enumerate(ALGO_NAMES):
        fig: plt.Figure
        fig, axs = plt.subplots(len(GRAPHS) // 2, 2, figsize=fig_size)

        fig.suptitle(algo_name + " scipy.sparse vs pycubool performance")
        fig.tight_layout()

        ax: plt.Axes
        for g_i, ax in enumerate(axs.flat):
            graph_name = GRAPHS[g_i].name
            graph_results = res[g_i]

            ax.set_xlabel(graph_name, weight="bold")
            ax.xaxis.set_label_position("top")

            ax.set_xticks(label_locations, REGEXES.keys())
            ax.tick_params(axis='x', which='both', bottom=False)
            if g_i % len(axs[0]) == 0:
                ax.set_ylabel("Time, s")

            scipy_means = [rs[0][algo_i][0] for rs in graph_results]
            scipy_stdevs = [rs[0][algo_i][1] for rs in graph_results]
            ax.bar(
                label_locations - bar_width / 2,
                scipy_means,
                bar_width,
                yerr=scipy_stdevs,
                color="tab:blue",
                label="scipy.sparse",
                capsize=cap_size,
            )

            pycubool_means = [rs[1][algo_i][0] for rs in graph_results]
            pycubool_stdevs = [rs[1][algo_i][1] for rs in graph_results]
            ax.bar(
                label_locations + bar_width / 2,
                pycubool_means,
                bar_width,
                yerr=pycubool_stdevs,
                color="tab:green",
                label="pycubool",
                capsize=cap_size,
            )

        handles, labels = axs[-1][-1].get_legend_handles_labels()
        fig.legend(handles, labels, bbox_to_anchor=(1, 1), loc="upper left")

    plt.show()


show_plots_1(fixed_starts_res)

### Результаты по вопросу 2 (о влиянии размера стартового множества на выгоду использования решений первой и третьей формулировок)

In [ ]:
def show_plots_2(res: list[list[dict[int, tuple[tuple[float, float], tuple[float, float]]]]]):
    fig: plt.Figure
    fig, axs = plt.subplots(len(GRAPHS), len(REGEXES), figsize=(5 * len(REGEXES), 2 * len(GRAPHS)))

    fig.suptitle("Start set size influence on algorithm 1 vs algorithm 3")
    fig.tight_layout()

    bar_width = 0.2
    cap_size = 4

    for g_i, axs_row in enumerate(axs):
        graph_name = GRAPHS[g_i].name
        graph_results = res[g_i]

        ax: plt.Axes
        for r_i, ax in enumerate(axs_row):
            regex_name = list(REGEXES.keys())[r_i]
            regex_results = graph_results[r_i]

            if g_i == 0:
                ax.set_xlabel(regex_name, weight="bold")
                ax.xaxis.set_label_position("top")

            if r_i == 0:
                ax.set_ylabel("Time, s")

            if r_i == len(REGEXES) - 1:
                ax.set_ylabel(graph_name, weight="bold")
                ax.yaxis.set_label_position("right")

            label_locations = np.arange(len(regex_results))
            ax.set_xticks(label_locations, regex_results.keys())

            algo1_results = [rs[0] for rs in regex_results.values()]
            algo1_means, algo1_stdevs = zip(*algo1_results)
            ax.bar(
                label_locations - bar_width / 2,
                algo1_means,
                bar_width,
                yerr=algo1_stdevs,
                color="tab:cyan",
                label=ALGO_NAMES[0],
                capsize=cap_size,
            )

            algo3_results = [rs[1] for rs in regex_results.values()]
            algo3_means, algo3_stdevs = zip(*algo3_results)
            ax.bar(
                label_locations + bar_width / 2,
                algo3_means,
                bar_width,
                yerr=algo3_stdevs,
                color="tab:orange",
                label=ALGO_NAMES[2],
                capsize=cap_size,
            )

            ax.tick_params(axis='x', which='both', bottom=False)

    handles, labels = axs[-1][-1].get_legend_handles_labels()
    fig.legend(handles, labels, bbox_to_anchor=(1, 1), loc="upper left")

    plt.show()


show_plots_2(varied_starts_res)

### Результаты по вопросу 3 (о сравнении быстродействия решений второй и третьей формулировок)

In [ ]:
def show_plots_3(
    res: list[list[tuple[tuple[tuple[float, float], tuple[float, float], tuple[float, float]], tuple[
        tuple[float, float], tuple[float, float], tuple[float, float]]]]]
):
    fig_size = (2.5 * len(REGEXES), len(GRAPHS))
    label_locations = np.arange(len(REGEXES))
    bar_width = 0.2
    cap_size = 4

    fig: plt.Figure
    fig, axs = plt.subplots(len(GRAPHS) // 2, 2, figsize=fig_size)

    fig.suptitle("BFS-based algorithms comparison")
    fig.tight_layout()

    ax: plt.Axes
    for g_i, ax in enumerate(axs.flat):
        graph_name = GRAPHS[g_i].name
        graph_results = res[g_i]

        ax.set_xlabel(graph_name, weight="bold")
        ax.xaxis.set_label_position("top")

        ax.set_xticks(label_locations, REGEXES.keys())
        ax.tick_params(axis='x', which='both', bottom=False)
        if g_i % len(axs[0]) == 0:
            ax.set_ylabel("Time, s")

        pycubool_algo2_means = [rs[1][1][0] for rs in graph_results]
        pycubool_algo2_stdevs = [rs[1][1][1] for rs in graph_results]
        ax.bar(
            label_locations - bar_width / 2,
            pycubool_algo2_means,
            bar_width,
            yerr=pycubool_algo2_stdevs,
            color="tab:purple",
            label=ALGO_NAMES[1],
            capsize=cap_size,
        )

        pycubool_algo3_means = [rs[1][2][0] for rs in graph_results]
        pycubool_algo3_stdevs = [rs[1][2][1] for rs in graph_results]
        ax.bar(
            label_locations + bar_width / 2,
            pycubool_algo3_means,
            bar_width,
            yerr=pycubool_algo3_stdevs,
            color="tab:orange",
            label=ALGO_NAMES[2],
            capsize=cap_size,
        )

    handles, labels = axs[-1][-1].get_legend_handles_labels()
    fig.legend(handles, labels, bbox_to_anchor=(1, 1), loc="upper left")

    plt.show()


show_plots_3(fixed_starts_res)

## Заключение

TODO